In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.9 MB/s 
     |████████████████████████████████| 6.6 MB 87.3 MB/s 


In [ ]:
import csv
import re
import itertools
import os

In [ ]:


"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
# https://www.reddit.com/r/datasets/comments/la6zuq/massive_multiturn_conversational_dataset_based_on/

# !unzip '/content/drive/MyDrive/Discord Chatbot/DIscord Conversation Data.zip'

In [ ]:
%cd '/content/drive/MyDrive/Discord Chatbot/'

/content/drive/MyDrive/Discord Chatbot


In [ ]:
versions = ['v1']
prefix1 = '/content/drive/MyDrive/Discord Chatbot/Discord_data/'
ver = ''
prefix2 = '/content/drive/Shareddrives/Datasets/cleaned-v4/'
          # v1/discord-v1-detox-antispam'

In [ ]:
def buildDataset():

  data = []
  #  \s* : 0 or more spaces; .* : 0 or more chars?
#  every () is a group
  regex ='(.*)(\s*:\s*)(.*)'
  eliminator = '(:)[a-z]+(:)'

  for v in versions:
    file_path = prefix1+v+prefix2+'discord-{0}-detox-antispam'.format(v)

    for file in os.listdir(file_path):
        filename = os.fsdecode(file)
        if filename.endswith(".txt"): 
          n = os.path.join(file_path, filename)
          print(n)
          with open(n, 'r') as in_file:
            texts = []
            for l in in_file:
              # to remove \t between participants in the original dataset
              # this actually separates the participants (at least from this sample)
              l = l.strip().split('\t')
              texts.append(l)
          #list of lists to a list
          merged = list(itertools.chain(*texts))


        # for k in keys:
        for text in merged:
          #use group to obtain correpsonding authors/msg (group1 (authors) group2 (:) group3 (msg))
          try: 
            s1 = re.match(regex, text).group(1)
          except AttributeError:
            print(text)
            continue
          s2 = re.match(regex, text).group(3).replace(',', '\,')
        #remove :smile: (emoji)
          s2 = re.sub(eliminator, "",s2)
        #proceed if s2 has not only emoji
          if s2:
            temp = [s1,s2]
            data.append(temp)
  print(len(data))
  return data

In [ ]:
data = buildDataset()

In [ ]:
csv_file = prefix1+'out_v1.csv'
keys = {'name': '', 'msg': ''}

with open(csv_file, 'w') as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL, escapechar='\\')
    writer.writerow(keys.keys())
    for i in range(len(data)):
        writer.writerow(data[i])

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(prefix1+'out_v1.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data[data['name'] == 'Petronilla'].head(10)

,name,msg
3304583,Petronilla,?mute\nAco is 800 animus nerd
3304585,Petronilla,"no nsfw in the chats\, thnx\nand memes go in d..."
3304599,Petronilla,Epic shitposter
3304618,Petronilla,that result will just show messages containing...
3304622,Petronilla,this weekend
3304650,Petronilla,Yup sao was boring
3304654,Petronilla,Exactly
3304666,Petronilla,Wrong
3304673,Petronilla,Have u ever played an actual MMO
3304700,Petronilla,@Pontifexstfu


In [ ]:
counts = data['name'].value_counts(dropna=False)
maps = data['name'].map(counts) 
aa = data[(maps >100) & (maps < 10000)]

In [ ]:
def removeMentions(txt):

  return re.sub(r'@\w+\s*', '', txt)

In [ ]:
aa.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
# names = aa['name'].unique()[:5000]
names = aa['name'].unique()[:1333]

compact_data = pd.DataFrame(columns = ['name', 'msg'])
count = aa['msg'].str.split().str.len()
# https://stackoverflow.com/questions/43943532/use-word-count-in-pandas-dataframe-to-drop-rows-with-only-one-word
sufficientMsg = aa[~(count < 4)]

# this one actually has the same effect
# sufficientMsg = aa[(aa['msg'].str.split().str.len() > 3)]


for name in names:
  # obtain messages that explicitly mention current 'name'
  # https://stackoverflow.com/questions/43727583/re-sub-erroring-with-expected-string-or-bytes-like-object
  name = str(name)
  if not (pd.isnull(name) or re.match(r'^[_\W]+$', name)):
    print(name)
    replies = sufficientMsg[(sufficientMsg["msg"].str.contains(fr'@\b{name}\b', regex=True, case=False)==True)]
    if(replies.shape[0] > 50):
      # remove all mentions, then if the msg has only @XXX, drop the msg.
      replies['msg'] = replies['msg'].apply(removeMentions)
      replies.drop(replies[replies.msg == ''].index, inplace=True)

      senders = sufficientMsg[(sufficientMsg['name']==name)]
      if(abs(senders.shape[0] - replies.shape[0]) < 2000):

        senders.loc[:, 'name'] = 'A'
        # senders['name'] = 'A'
        # replies['name'] = 'B'
        replies.loc[:, 'name'] = 'B'
        temp_df = pd.concat([replies, senders])
        temp_df.sort_index(inplace=True)
        compact_data = pd.concat([compact_data, temp_df], ignore_index = True)

In [ ]:
import string

In [ ]:
compact_data['name'].value_counts()

A    802840
B    338749
Name: name, dtype: int64

In [ ]:
# compact_data.to_csv(prefix1+'new_compact_data_1.csv', encoding='utf-8', index=False)
compact_data.to_csv(prefix1+'1333_new_compact_data_1.csv', encoding='utf-8', index=False)

In [ ]:
# compact_data.to_csv(prefix1+'compact_data_1.csv', encoding='utf-8', index=False)

In [ ]:
compact_data = pd.read_csv(prefix1+'new_compact_data_1.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# data.shape
# d = data.head(300000)
# d.to_csv(prefix1+'out_sliced.csv', encoding='utf-8', index=False)

(81869576, 2)

In [ ]:
compact_data.head(20)

,name,msg
0,A,'sup nib nobs
1,A,what if I told you\ni made this
2,A,im tryna buy verge\nhow do I buy verge
3,A,I popped in from the IRC channel since people ...
4,A,@Ruru Cinnamon92
5,A,yeah haven't seen you talk in the IRC side of ...
6,A,"prolly gonna be interesting this\, some overla..."
7,A,"I mean I read the rules but anyway like\, ever..."
8,A,"aight that's cool\, thanks @coffee"
9,A,I'm kinda miffed someone is/u/satan and isn't ...


In [ ]:
compact_data['name'].value_counts()

A    4301729
B    1203212
Name: name, dtype: int64

In [ ]:
test = compact_data.iloc[:2000]

In [ ]:
import numpy as np

n = 8
l=[test.iloc[x,0] in test.iloc[x-n:x,0].tolist() for x in np.arange(n,len(test))]


In [ ]:
contexts = []

n = 8

for i in range(n, len(compact_data['msg'])):
  row = []
  prev = i - 1 - n # we additionally substract 1, so row will contain current responce and 7 previous responces  
  for j in range(i, prev, -1):
    row.append(compact_data['msg'][j])
  contexts.append(row)

In [ ]:
len(contexts)

228540

In [ ]:
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]

In [ ]:
df = pd.DataFrame.from_records(contexts, columns=columns)
df.head(5)

,response,context,context/0,context/1,context/2,context/3,context/4,context/5,context/6
0,i think that's the brief synopsis of black mir...,40 gallon water changes twice a week NO THANK YOU,i don't miss having a tiger oscar or a 75 gall...,"In 2004\, Hasbro collaborated with Takara and ...",why didn't we have this when i was a kid?,so i just found out about these,who wants to play some beyblade,who wants to see my grandmas dog,this thing has me DYING
1,@jarric i used a fluval fx6 and two cheaper ca...,i think that's the brief synopsis of black mir...,40 gallon water changes twice a week NO THANK YOU,i don't miss having a tiger oscar or a 75 gall...,"In 2004\, Hasbro collaborated with Takara and ...",why didn't we have this when i was a kid?,so i just found out about these,who wants to play some beyblade,who wants to see my grandmas dog
2,the fx6 is an annoying filter,@jarric i used a fluval fx6 and two cheaper ca...,i think that's the brief synopsis of black mir...,40 gallon water changes twice a week NO THANK YOU,i don't miss having a tiger oscar or a 75 gall...,"In 2004\, Hasbro collaborated with Takara and ...",why didn't we have this when i was a kid?,so i just found out about these,who wants to play some beyblade
3,it's S300 and if the tiniest bit of sand gets ...,the fx6 is an annoying filter,@jarric i used a fluval fx6 and two cheaper ca...,i think that's the brief synopsis of black mir...,40 gallon water changes twice a week NO THANK YOU,i don't miss having a tiger oscar or a 75 gall...,"In 2004\, Hasbro collaborated with Takara and ...",why didn't we have this when i was a kid?,so i just found out about these
4,then you have to drain the filter,it's S300 and if the tiniest bit of sand gets ...,the fx6 is an annoying filter,@jarric i used a fluval fx6 and two cheaper ca...,i think that's the brief synopsis of black mir...,40 gallon water changes twice a week NO THANK YOU,i don't miss having a tiger oscar or a 75 gall...,"In 2004\, Hasbro collaborated with Takara and ...",why didn't we have this when i was a kid?


In [ ]:
df.to_csv(prefix1+'1333_new_df_1.csv', encoding='utf-8', index=False)

In [ ]:
# df = pd.read_csv(prefix1+'new_df_1.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
import pandas as pd
df = pd.read_csv(prefix1+'1333_new_df_1.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
trn_df, val_df = train_test_split(df, test_size = 0.1, random_state = 42)

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)
        # https://github.com/allenai/openpi-dataset/issues/6#issuecomment-745412790
        if inputs.shape[1] > 1024: continue

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 64
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 2
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = False
        self.should_continue = True
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [ ]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]
            print(args.model_name_or_path)

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(trn_df, val_df)

06/23/2022 21:30:01 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False


Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

06/24/2022 00:24:00 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-105000
06/24/2022 00:24:02 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-105000
06/24/2022 00:29:32 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-108500
06/24/2022 00:29:37 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-108500
06/24/2022 00:35:08 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-112000
06/24/2022 00:35:11 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-112000
06/24/2022 00:40:39 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-115500
06/24/2022 00:40:45 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-115500
06/24/2022 00:46:14 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-119000
06/24/2022 00:46:17 - INFO - __main__ -   Sa

Iteration:   0%|          | 0/102843 [00:00<?, ?it/s]

06/24/2022 03:05:22 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-206500
06/24/2022 03:05:25 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-206500
06/24/2022 03:10:54 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-210000
06/24/2022 03:10:57 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-210000
06/24/2022 03:16:26 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-213500
06/24/2022 03:16:31 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-213500
06/24/2022 03:22:00 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-217000
06/24/2022 03:22:03 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-217000
06/24/2022 03:27:33 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-220500
06/24/2022 03:27:36 - INFO - __main__ -   Sa

Evaluating:   0%|          | 0/11427 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
main(trn_df, val_df)

06/24/2022 16:15:18 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False


output-small/checkpoint-308000


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
06/24/2022 16:15:24 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7fd5e86ce890>
06/24/2022 16:15:24 - INFO - __main__ -   Loading features from cached file cached/gpt2_cached_lm_512
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
06/24/2022 16:15:28 - INFO - __main__ -   ***** Running training *****
06/24/2022 16:15:28 - INFO - __m

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/102843 [00:00<?, ?it/s]

06/24/2022 16:16:06 - INFO - __main__ -    global_step = 308529, average loss = 0.00345351436143261
06/24/2022 16:16:06 - INFO - __main__ -   Saving model checkpoint to output-small
06/24/2022 16:16:12 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small']
06/24/2022 16:16:14 - INFO - __main__ -   Loading features from cached file cached/gpt2_cached_lm_512
06/24/2022 16:16:17 - INFO - __main__ -   ***** Running evaluation  *****
06/24/2022 16:16:17 - INFO - __main__ -     Num examples = 205686
06/24/2022 16:16:17 - INFO - __main__ -     Batch size = 4


Evaluating:   0%|          | 0/51421 [00:00<?, ?it/s]

06/24/2022 16:36:46 - INFO - __main__ -   ***** Eval results  *****
06/24/2022 16:36:46 - INFO - __main__ -     perplexity = tensor(3.7497)


{'perplexity_': tensor(3.7497)}

In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/Discord Chatbot/output-small')

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
model.push_to_hub('discord_test')

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='discord_test' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Gods/discord_test into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/487M [00:00<?, ?B/s]

To https://huggingface.co/Gods/discord_test
   a0e9646..5d114e7  main -> main



'https://huggingface.co/Gods/discord_test/commit/5d114e7bd795aafab81a5ff5e86788677e341286'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Discord Chatbot/output-small')

In [ ]:
tokenizer.push_to_hub('discord_test')

To https://huggingface.co/Gods/discord_test
   5d114e7..787e73c  main -> main



'https://huggingface.co/Gods/discord_test/commit/787e73cc2b5e10ab663f407787042253828a4aa2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)

In [ ]:
tokenizer.model_max_length

1024